In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import scipy.stats as stats
import scipy.linalg

# Fix a random seed (if necessary) to get same random output
np.random.seed(21)

# Function to download stock data from Yahoo Finance and calculate returns
def download_stock_data(ticker_set, start_date, end_date):
    stock_data = yf.download(ticker_set, start=start_date, end=end_date)['Adj Close']
    stock_return=np.log(stock_data/stock_data.shift(1)).dropna()  # We consider the log return
    return stock_return

# Function to calculate portfolio metrics (mean, variance, and covariance matrix)
def portfolio_metrics(stock_return, weights):
    portfolio_mean = np.dot(stock_return.mean(), weights)
    cov_matrix = stock_return.cov()
    portfolio_var = np.dot(weights, np.dot(cov_matrix, weights.T))
    return portfolio_mean, portfolio_var, cov_matrix

# Function to calculate VaR using Historical Simulation
def calculate_var_historical(portfolio_return, confidence_intervals):
    sorted_returns = np.sort(portfolio_return)
    VaR_hist=[np.percentile(sorted_returns,100-ci) for ci in confidence_intervals]
    return VaR_hist

# Function to calculate VaR using Variance-Covariance method
def calculate_var_var_cov(portfolio_mean, portfolio_var, confidence_intervals):
    VaR_var_cov = [stats.norm.ppf(1 - ci/100) * np.sqrt(portfolio_var) + portfolio_mean for ci in confidence_intervals]
    return VaR_var_cov

# Function to calculate VaR using Monte Carlo Simulation
def calculate_var_monte_carlo(stock_return, weights, confidence_intervals, num_simulations=100000):
    drift = stock_return.mean() - (stock_return.var() / 2)
    volatility = stock_return.std()
    
    # Generate correlated random returns using Cholesky decomposition
    corr_matrix = stock_return.corr()
    cholesky_factor = scipy.linalg.cholesky(corr_matrix, lower=True)
    
    random_returns = np.random.normal(size=(len(weights), num_simulations))
    correlated_random_returns = np.dot(cholesky_factor, random_returns)
    
    simulated_returns = drift.values[:, np.newaxis] + volatility.values[:, np.newaxis] * correlated_random_returns
    simulated_portfolio_returns = np.dot(weights, simulated_returns)
    
    simulated_portfolio_returns.sort()
    VaR_mc = [np.percentile(simulated_portfolio_returns,100-ci) for ci in confidence_intervals]
    return VaR_mc

# Main function to run the VaR analysis
def main():
    # Define parameters
    #ticker_set : NIFTY 50 stocks are taken
    ticker_set=[
    "ASIANPAINT.NS", "BRITANNIA.NS", "CIPLA.NS", "EICHERMOT.NS", "NESTLEIND.NS",
    "GRASIM.NS", "HEROMOTOCO.NS", "HINDALCO.NS", "HINDUNILVR.NS", "ITC.NS",
    "LT.NS", "M&M.NS", "RELIANCE.NS", "TATACONSUM.NS", "TATAMOTORS.NS",
    "TATASTEEL.NS", "WIPRO.NS", "APOLLOHOSP.NS", "DRREDDY.NS", "TITAN.NS",
    "SBIN.NS", "SHRIRAMFIN.NS", "BPCL.NS", "KOTAKBANK.NS", "INFY.NS",
    "BAJFINANCE.NS", "ADANIENT.NS", "SUNPHARMA.NS", "JSWSTEEL.NS", "HDFCBANK.NS",
    "TCS.NS", "ICICIBANK.NS", "POWERGRID.NS", "MARUTI.NS", "INDUSINDBK.NS",
    "AXISBANK.NS", "HCLTECH.NS", "ONGC.NS", "NTPC.NS", "COALINDIA.NS",
    "BHARTIARTL.NS", "TECHM.NS", "LTIM.NS", "DIVISLAB.NS", "ADANIPORTS.NS",
    "HDFCLIFE.NS", "SBILIFE.NS", "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "BAJAJFINSV.NS"
]
    start_date = "2020-01-01"  #Date format is "YYYY-MM-DD"
    end_date = "2024-01-01"
    weights = np.random.random(len(ticker_set))
    weights /= weights.sum()  # Normalize weights
    confidence_intervals = [90, 95, 99]

    # Get stock data and calculate returns
    stock_return = download_stock_data(ticker_set, start_date, end_date)

    # Calculate portfolio metrics
    portfolio_mean, portfolio_var, _ = portfolio_metrics(stock_return, weights)
    
    # Calculate portfolio returns
    portfolio_return = np.dot(stock_return, weights)
    
    # Calculate VaR using different methods
    VaR_hist = calculate_var_historical(portfolio_return, confidence_intervals)
    VaR_var_cov = calculate_var_var_cov(portfolio_mean, portfolio_var, confidence_intervals)
    VaR_mc = calculate_var_monte_carlo(stock_return, weights, confidence_intervals)
    
    # Compile results into a DataFrame
    VaR_results = pd.DataFrame(
        [VaR_hist, VaR_var_cov, VaR_mc],
        columns=["90%", "95%", "99%"],
        index=["Historical Method", "Var-Cov Method", "Monte Carlo Method"]
    )
    
    print("1 Day Value at Risk (VaR) Results:")
    print(VaR_results)

# Run the main function
if __name__ == "__main__":
    main()


[*********************100%%**********************]  50 of 50 completed


1 Day Value at Risk (VaR) Results:
                         90%       95%       99%
Historical Method  -0.011558 -0.017938 -0.040642
Var-Cov Method     -0.015641 -0.020317 -0.029089
Monte Carlo Method -0.015917 -0.020657 -0.029325
